In [2]:
import numpy as np
from ape import chain, Contract, networks

In [3]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [4]:
mim_curve_pool = Contract("0x5a6A4D54456819380173272A5E8E9B9904BdF41B")
mim_curve_pool

<Vyper_contract 0x5a6A4D54456819380173272A5E8E9B9904BdF41B>

In [5]:
base_curve_pool = Contract("0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7")
base_curve_pool

<Vyper_contract 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7>

In [6]:
# A = 2000
A = mim_curve_pool.A()
A

2000

In [7]:
PRECISION = 10**18
total_supply = mim_curve_pool.totalSupply()
D = mim_curve_pool.get_virtual_price() * total_supply / PRECISION

In [8]:
# D ~ 98.534M
D /= PRECISION
D

98533841.08789892

In [9]:
# get the balances for mim and 3crv
# mim is coin0, 3crv is coin1
coin0_addr = mim_curve_pool.coins(0)
coin1_addr = mim_curve_pool.coins(1)

print(f"coin 0: {coin0_addr}")
print(f"coin 1: {coin1_addr}")

coin 0: 0x99D8a9C45b2ecA8864373A26D1459e3Dff1e17F3
coin 1: 0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490


In [10]:
mim = Contract(coin0_addr)
mim

<MagicInternetMoneyV1 0x99D8a9C45b2ecA8864373A26D1459e3Dff1e17F3>

In [11]:
crv3 = Contract(coin1_addr)
crv3

<3Crv 0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490>

In [12]:
bal_mim = mim_curve_pool.balances(0) / (10 ** mim.decimals()) # adjust for decimals
bal_3crv = mim_curve_pool.balances(1) / (10 ** crv3.decimals())

print(f"balance 0 (MIM): {bal_mim}")
print(f"balance 1 (3Crv): {bal_3crv}")

balance 0 (MIM): 58804981.915712215
balance 1 (3Crv): 38843750.143145576


In [13]:
# some stableswap fns for two coin case
def _d():
    return bal_mim + bal_3crv

def B(x):
    return _d() * (1 - 4*A) * x + 4*A*(x**2)

def E(x):
    return 4*A*x

def y(x):
    return (-B(x) + np.sqrt(B(x)**2 + (_d()**3) * E(x))) / (2 * E(x))

def p(x):
    return (y(x) / x) * ((_d()**3) + 16 * A * (x**2) * (y(x))) / ((_d()**3) + 16 * A * (x) * (y(x)**2))

In [14]:
# about in line with current balance of 3crv in pool; diff is about .137 bps
print('y(bal_mim):', y(bal_mim))
print('y(bal_mim) / bal_3crv - 1:', y(bal_mim) / bal_3crv - 1)

y(bal_mim): 38844282.268111326
y(bal_mim) / bal_3crv - 1: 1.3699114111975774e-05
